In [ ]:
import numpy as np
import pandas as pd
import cv2

In [ ]:
tt=pd.read_json("/Users/souvicchakraborty/Desktop/kaggle/iceberg/train.json")

In [ ]:
tt.keys()

In [ ]:
ii1=np.array([np.array(i).reshape((75,75)) for i in tt['band_1']]).reshape((-1,75,75,1))

In [ ]:
ii2=np.array([np.array(i).reshape((75,75)) for i in tt['band_2']]).reshape((-1,75,75,1))

In [ ]:
ii=np.concatenate((ii1,ii2),axis=3)

In [ ]:
ii=ii[indices,:,:,:]+adder[:,None,None,None]

In [ ]:
means=np.mean(ii,axis=(0,1,2))

In [ ]:
stds=np.std(ii,axis=(0,1,2))

In [ ]:
stds

In [ ]:
ii3=(ii-means)/stds

In [ ]:
ii3.shape

In [ ]:
trainindices=np.random.choice(1604,1200,replace=False)

In [ ]:
testindices=[i for i in range(1604) if i not in trainindices]

In [ ]:
yy=tt['is_iceberg'][indices]

In [ ]:
from keras.layers import Input, Flatten, Dense
from keras.models import Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [ ]:
from keras.layers.normalization import BatchNormalization
import time

In [ ]:
import keras

In [ ]:
model = Sequential()
batch_size=1024
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(16, (1, 1), batch_input_shape=(None,75, 75,2),padding='same',use_bias=True))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3),padding='same',use_bias=True))
model.add(Activation('elu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(64, (1, 1),padding='same',use_bias=True))
model.add(Activation('elu'))

model.add(Conv2D(128, (3, 3),padding='same',use_bias=True))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (1, 1),padding='same',use_bias=True))
model.add(Activation('elu'))
model.add(Conv2D(128, (3, 3),padding='same',use_bias=True))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dropout(0.75))

#model.add(Conv2D(64, (3, 3),padding='same',use_bias=False))
#model.add(BatchNormalization())
#model.add(Activation('relu'))

#model.add(MaxPooling2D(pool_size=(3, 3)))

#model.add(Conv2D(16, (3, 3), activation='relu',padding='same'))
#model.add(Conv2D(16, (3, 3), activation='relu',padding='same',use_bias=True))
#model.add(MaxPooling2D(pool_size=(4, 4)))
#model.add(Dropout(0.25))
#model.add(Dropout(0.25))
#model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
#model.add(Conv2D(16, (3, 3), activation='relu',padding='same',use_bias=True))
#model.add(MaxPooling2D(pool_size=(4, 4)))
#model.add(Dropout(0.25))


model.add(Dense(64,use_bias=True))
model.add(Activation('elu'))
model.add(Dropout(0.25))
model.add(Dense(64,use_bias=True))
model.add(Activation('elu'))
model.add(Dropout(0.5))

model.add(Dense(1,use_bias=True))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.05),#'sgd',keras.optimizers.SGD(lr=0.05),
              metrics=['accuracy'])

In [ ]:
model.fit(ii3, yy, epochs=200, batch_size=8,validation_split=0.1)

In [ ]:
model.evaluate(ii3[trainindices,:,:,:], yy[trainindices], batch_size=200)

In [ ]:
model.evaluate(ii3[testindices,:,:,:], yy[testindices], batch_size=200)

In [ ]:
model.fit(ii3, yy, epochs=200, batch_size=32,validation_split=0.2)

In [ ]:
ii4=np.concatenate([ii3,ii3[:,:,::-1,:]],axis=0);yy2=np.array(yy.tolist()+yy.tolist())

In [ ]:
ii5=np.concatenate([ii4,ii4[:,::-1,:,:]],axis=0);yy5=np.array(yy2.tolist()+yy2.tolist())

In [ ]:
ii6=np.concatenate([ii5,ii5.transpose((0,2,1,3))],axis=0);yy6=np.array(yy5.tolist()+yy5.tolist())

In [ ]:
model.fit(ii6, yy6, epochs=50, batch_size=8,validation_split=0.1)

In [ ]:
model.fit(ii6, yy6, epochs=50, batch_size=16,validation_split=0.2)

In [ ]:
tt.keys()

In [ ]:
[np.cos(np.pi*cd/180.0) for cd in np.array(tt['inc_angle']).flatten()]

In [ ]:
np.cos(np.pi)

In [ ]:
tt['inc_angle'][0]

In [ ]:
np.cos(np.pi*43.9239/180.0)

In [ ]:
tt['inc_angle'].shape

In [ ]:
np.sum(type(tt['inc_angle'])==float)

In [ ]:
type(tt['inc_angle'][1])==float

In [ ]:
np.isnan(np.array(tt['inc_angle']))

In [ ]:
indices=np.argwhere(np.array([tr!='na' for tr in tt['inc_angle']])).flatten()

In [ ]:
gs=((tt['inc_angle'][indices]))

In [ ]:
cosg=[np.cos(np.pi/180.0*g) for g in gs]

In [ ]:
len(cosg)

In [ ]:
adder=10*np.log10(cosg)

In [ ]:
testi=pd.read_json("../../kaggle/iceberg/test.json")

In [ ]:
tii1=np.array([np.array(i).reshape((75,75)) for i in testi['band_1']]).reshape((-1,75,75,1))
tii2=np.array([np.array(i).reshape((75,75)) for i in testi['band_2']]).reshape((-1,75,75,1))

In [ ]:
tii=np.concatenate((tii1,tii2),axis=3)

In [ ]:
tcosg=[np.cos(np.pi/180.0*g) for g in testi['inc_angle']]

In [ ]:
tadder=10*np.log10(tcosg)

In [ ]:
tii=tii+tadder[:,None,None,None]

In [ ]:
tii3=(tii-means)/stds

In [ ]:
pii6=model.predict(tii3)

In [ ]:
pii3.shape

In [ ]:
ddw=pd.read_csv("../../kaggle/iceberg/sample_submission.csv")

In [ ]:
ddw[:10]

In [ ]:
ddw['is_iceberg']=piio#[:,0]

In [ ]:
ddw.to_csv("../../kaggle/iceberg/ss7.csv",index=False)

In [ ]:
ii4.shape

In [ ]:
piio=0.3*pii6[:,0]+0.7*pii4[:,0]

In [ ]:
piio[np.argwhere(piio>1.0)]=1

In [ ]:
del(model)